# Imports and creation of dataframes

In [1]:
import pandas as pd
import re

# PL league standings 2010-2022
for year in range(2010, 2023, 1):
    locals()['dftable_{}'.format(year)] = pd.read_csv('tables 2010+/'+str(year)+'.csv')

# All transfers from  1992 onwards
df_transfers = pd.read_csv('premier-league.csv')

# PL wages 2013-2022
for year in range(2014, 2023, 1):
    locals()['dfwage_{}'.format(year)] = pd.read_csv('wages/wages_'+str(year-1)+'.csv', index_col=0)

# PL team value 2010-2022
for year in range(2010, 2023, 1):
    locals()['dfvalue_{}'.format(year)] = pd.read_csv('teamvalue/teamvalue_'+str(year-1)+'.csv', index_col=0)

# PL attendance 2010-2022
for year in range(2010, 2024, 1):
    locals()['dfatt_{}'.format(year)] = pd.read_csv('attendance/attendance_'+str(year)+'.csv', index_col=0)

# Structuring of 'dataframes'
###### Transfers: We only want total spend/net spend as an int, per team in separate df for each season
###### Attendance: We only want the percentage full as an int, per team in separate df for each season
###### Wages: We only want the £ value as an int, per team in separate df for each season
###### TeamValue: We only want the value as an int, per team in separate df for each season

In [2]:
# TRANSFERS
# Set new column with the year season ended
df_transfers['season_end'] = df_transfers['season'].str[5:]

def net_spend(df):
    """
    Function to transform the dataframe containing every transfer for every club across a single season into the total spend and net spend for each club during that season.
    :param the original dataframe:
    :return the new dataframe:
    """

    # Sum the incoming and outgoing transfer fees for each club
    df_slim = df.groupby(['club_name','transfer_movement']).sum('fee_cleaned').reset_index()[['club_name','transfer_movement','fee_cleaned']]
    # Calculate the difference in a new column, 'net_spend'
    df_slim['net_spend'] = df_slim['fee_cleaned']-df_slim['fee_cleaned'].shift(-1)
    # Tidy up the table: rename/remove redundant rows/cols, take 'FC' out of the team names.
    df_slim = df_slim[::2].reset_index()
    df_slim.rename(columns={'fee_cleaned':'total_spend','club_name':'Team'}, inplace=True)
    df_slim.drop(columns=['index','transfer_movement'], inplace=True)
    # df_slim['Team'] = df_slim['Team'].apply(tidy_names)
    return df_slim

# Cycle through each year(season), create a df for each season and transform to net spend
for year in range(1993, 2023, 1):
    locals()['dftransfer_{}'.format(year)] = df_transfers[df_transfers['season_end']==str(year)]
    locals()['dftransfer_{}'.format(year)] = net_spend(locals()['dftransfer_{}'.format(year)])

# ATTENDANCE
#Calculate percentage attendance in new column, then remove useless columns
for year in range(2010, 2023, 1):
    dframe = locals()['dfatt_{}'.format(year)]
    dframe['att_percent'] = dframe['av_attendance']/dframe['capacity']
    dframe.drop(columns=['capacity','av_attendance'], inplace=True)

# WAGES
# Use regex to tidy up value
def wage_int(val):
    val = re.sub(r'£ ', '', val)
    val = re.sub(r' \(.*', '', val)
    val = re.sub(r',', '', val)
    return int(val)
# Apply function to dframes
for year in range(2014, 2023, 1):
    dframe = locals()['dfwage_{}'.format(year)]
    dframe['annual_wage'] = dframe['annual_wage'].apply(wage_int)

# TEAM
# # Use regex to tidy up value
def value_int(val):
    val = re.sub(r'€', '', val)
    if bool(re.match(r'.*bn',val)):
        val = re.sub(r'bn', '', val)
        val = float(val) * 1000000000
    else:
        val = re.sub(r'm', '', val)
        val = float(val)* 1000000
    return int(val)
# # Apply function to dframes
for year in range(2011, 2023, 1):
    dframe = locals()['dfvalue_{}'.format(year)]
    dframe['TotValue'] = dframe['TotValue'].apply(value_int)

# Merge the datasets
##### Prepare main table (add a target value, streamline the features)
##### Merge with other tables one by one

In [3]:
# Standardise the team names in the tables
def tidy_names(name):
    name = re.sub(r'.FC', '', name)
    name = re.sub(r' $', '', name)
    name = re.sub(r'^ ', '', name)
    name = re.sub(r'Utd', 'United', name)
    name = re.sub(r'^Tottenham$', 'Tottenham Hotspur', name)
    name = re.sub(r'^West Brom$', 'West Bromwich Albion', name)
    name = re.sub(r'^West Ham$', 'West Ham United', name)
    name = re.sub(r'^Wolves$', 'Wolverhampton Wanderers', name)
    name = re.sub(r'^Brighton$', 'Brighton & Hove Albion', name)
    name = re.sub(r'^QPR$', 'Queens Park Rangers', name)
    name = re.sub(r'^Huddersfield$', 'Huddersfield Town', name)
    return name

def set_abb(name):

    if 'Man' or 'Wes' in name:
        if 'City' in name:
            abb = ('MCI')
        elif 'ches' in name:
            abb = ('MNU')
        elif 'Ham' in name:
            abb = ('WHU')
        else:
            abb = ('WBA')
    else:
        abb = name[:3].upper()
    return abb

#Prep the main table (league table)
for year in range(2010, 2023, 1):

    dframe = locals()['dftable_{}'.format(year)]

    # Tidy names and create abbreviations
    dframe['Team'] = dframe['Team'].apply(tidy_names)
    dframe['abb'] = dframe['Team'].apply(set_abb)

    # Set the top6 target for visualisation purposes
    top6 = ['Manchester United', 'Manchester City', 'Liverpool', 'Arsenal', 'Chelsea', 'Tottenham Hotspur']
    def set_target(team):
        if team in top6:
            target = 1
        else:
            target = 0
        return target
    dframe['target'] = dframe['Team'].apply(set_target)

    # Drop redundant columns
    dframe.drop(columns=['Pos','W','D','L','GF','GA','GD'], inplace=True)

# Merging function, standardises the team names
def standardise_merge(mainframe, df_add):

        # Standardise the team names
        df_add['Team'] = df_add['Team'].apply(tidy_names)

        # Merge the league table and transfer data
        return mainframe.merge(df_add, on='Team')

In [4]:
# For each season, merge the dataframes and export

for year in range(2014, 2023, 1):

    locals()[f'df_{year}'] = standardise_merge(locals()[f'dftable_{year}'], locals()[f'dfatt_{year}'])
    locals()[f'df_{year}'] = standardise_merge(locals()[f'df_{year}'], locals()[f'dfvalue_{year}'])
    locals()[f'df_{year}'] = standardise_merge(locals()[f'df_{year}'], locals()[f'dfwage_{year}'])
    globals()[f'df_{year}'] = standardise_merge(locals()[f'df_{year}'], locals()[f'dftransfer_{year}'])
    # print(len(locals()[f'df_{year}']))
    # print(locals()[f'df_{year}'].isnull().sum())
    globals()[f'df_{year}'].to_csv(f'clubsize_{year}.csv')